In [1]:
#  breakhistf2v026-nsl-v4-02.ipynb  v1.0.0 
# 
#  --------------------------------------------------
#  Hangzhou Domain Zones Technology Co., Ltd

#  Apache Licence 2.0       https://www.apache.org/licenses/LICENSE-2.0
#  --------------------------------------------------


import os
import tensorflow as tf
import numpy as np
import csv
import matplotlib.pyplot as plt
from tensorflow import keras
import neural_structured_learning as nsl
from tensorflow.python.keras.api._v2.keras import layers, optimizers, losses
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
import random


tf.random.set_seed(22)
np.random.seed(22)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')

IMAGE_INPUT_NAME = 'images'
LABEL_INPUT_NAME = 'labels'
  
%matplotlib inline
 
def read_csv(csvnamepath, filename, is2=False):
     

    # read from csv file
    images, labels = [], []
    benigns = ["adenosis" ,"fibroadenoma" ,"tubular_adenoma" , "phyllodes_tumor"]
    with open(os.path.join(csvnamepath, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label, cancername = row
            if is2:
                if  cancername in benigns:
                    label = 0
                else:
                    label = 1
            else:
                label = int(label)

            images.append(img)
            labels.append(label)

    assert len(images) == len(labels)

    return images, labels

@tf.function 
def load_breakhis(dirc, filename, is2 = False , mode='train' ):
    classdir2label={}
     
    filedirs = os.listdir( dirc)
    for filedir in filedirs:
        if not os.path.isdir(os.path.join(dirc,filedir)):
            continue
        classdir2label[filedir]=len(classdir2label.keys())
    

     
    images, labels = read_csv(os.path.join(os.path.abspath('.'),'tf2breakhis'), filename ,is2)  
    if mode == 'train':  # 60%
        images = images[:int(0.6 * len(images))]
        labels = labels[:int(0.6 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # 20% = 80%->100%
        images = images[int(0.8 * len(images)):]
        labels = labels[int(0.8 * len(labels)):]
    return images, labels, classdir2label
 
img_mean = tf.constant([0.485, 0.456, 0.406])
img_std = tf.constant([0.229, 0.224, 0.225])

@tf.function
def normalize(x, mean=img_mean, std=img_std):
    # x shape: [224, 224, 3]
    # mean：shape为1；这里用到了广播机制。我们安装好右边对齐的原则，可以得到如下；
    # mean : [1, 1, 3], std: [3]        先插入1
    # mean : [224, 224, 3], std: [3]    再变为224
    x = (x - mean)/std
    return x

# 数据normalize之后，这里有一个反normalizaion操作。比如数据可视化的时候，需要反过来。
# @tf.function
def denormalize(x, mean=img_mean, std=img_std):
    x = x * std + mean
    return x

  
# @tf.function 
def preprocess_train_nsl(images,labels):
    i =0;
    images1= []
    labels1= []
    imagelen = len(images)
    #imagelen = 5
    print (tf.__version__)
    print (len(images))
    while (True):
        if i >= imagelen :
            break
        
        x= images[i]
        y =labels[i]
        x = tf.io.read_file(x)
        x = tf.image.decode_jpeg(x, channels=3) # RGBA    
        x = tf.image.resize(x, [244, 244])
        
        x1 = tf.image.random_flip_left_right(x)
     
        # x = tf.image.random_flip_up_down(x)
         
        x = tf.image.random_crop(x, [224,224,3])
        
        x1 = tf.image.random_crop(x1, [224,224,3])
        
       # x = tf.image.resize_with_crop_or_pad(x, 460,700)
        #x = tf.image.random_flip_left_right(x)
       #   x = tf.image.random_crop(x, [460,700,3])
        x = tf.cast(x, dtype=tf.float32) / 255.
        x = normalize(x)
        x = tf.convert_to_tensor(x) 
        
        x1 = tf.cast(x1, dtype=tf.float32) / 255.
        x1 = normalize(x1)
        x1 = tf.convert_to_tensor(x1) 
        
        y = tf.convert_to_tensor(y)
        #y = tf.one_hot(y, depth=8)      
        images1.append(x)
        labels1.append(y)
        
        images1.append(x1)
        labels1.append(y)
        
        i = i+1 
    return images1, labels1


def preprocessnsl(images,labels):
    i =0;
    images1= []
    labels1= []
    imagelen = len(images)
    #imagelen = 5
    print (tf.__version__)
    print (len(images))
    while (True):
        if i >= imagelen :
            break
        
        x= images[i]
        y =labels[i]
        x = tf.io.read_file(x)
        x = tf.image.decode_jpeg(x, channels=3) # RGBA    
        x = tf.image.resize(x, [244, 244])
   
        x = tf.image.random_crop(x, [224,224,3])
       # x = tf.image.resize_with_crop_or_pad(x, 460,700)
        #x = tf.image.random_flip_left_right(x)
       #   x = tf.image.random_crop(x, [460,700,3])
        x = tf.cast(x, dtype=tf.float32) / 255.
        x = normalize(x)
        x = tf.convert_to_tensor(x) 
        y = tf.convert_to_tensor(y)
        #y = tf.one_hot(y, depth=8)      
        images1.append(x)
        labels1.append(y)
        i = i+1 
    return images1, labels1




def convert_to_tensor(images,labels):
    timages1 = tf.convert_to_tensor(images)
    #timages1.reshape(imagelen,2224,224,3) 
    tlabels1 = tf.convert_to_tensor(labels) 
    #tlabels1 = tf.one_hot(tlabels1, depth=8)
    return timages1, tlabels1

 
def shuffletwolists(list1,list2,shufflesize):
    temp = list(zip(list1, list2))
    i =0
    while(True):
        if i > shufflesize:
            break  
        random.shuffle(temp)
        i = i+1
    
    res1 ,res2 =zip(*temp)
    return list(res1), list(res2)



@tf.function 
def convert_to_dictionaries(image, label):
    return {'images':image, 'labels':label}

 
  
  

batch_size = 32
 
    
images, labels, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'),  "tf2breakhisnewCSV",False,'train')


(images , labels)  =shuffletwolists( images ,labels ,1000)  

(x_train , y_train)  = preprocess_train_nsl(images ,labels ) 

 
(x_train , y_train)  =shuffletwolists( x_train ,y_train ,1000)

  
images2, labels2, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'),  "tf2breakhisnewCSV",False,'val')
val_steps =  len(images2) / batch_size

 

(x_val , y_val)  = preprocessnsl(images2 ,labels2 ) 

(x_val , y_val)  =shuffletwolists( x_val ,y_val ,1000 ) 
 

db_train = tf.data.Dataset.from_tensor_slices({'feature':x_train, 'label':y_train}).batch(batch_size)
 
db_val =  tf.data.Dataset.from_tensor_slices({'feature':x_val, 'label':y_val}).batch(batch_size)

  

resnetdense = keras.Sequential([

    tf.keras.Input((224, 224,3), name='feature'),
    #tf.keras.Input((460, 700,3), name='feature'),
    
    #tf.keras.Input((456, 692,3), name='feature'),
 
    
    tf.keras.layers.Conv2D(16,5,3 , activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(3,3),
    #tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(64,5,3, activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(2,2),
    #tf.keras.layers.ReLU(),
    
    tf.keras.layers.Flatten(),
   
    #tf.keras.layers.Dense(128), 
    
    tf.keras.layers.Dense(128,activation=tf.nn.sigmoid ), 
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),  
    #tf.keras.layers.ReLU(),
    #tf.keras.layers.Dense(8)
    tf.keras.layers.Dense(8, activation=tf.nn.softmax)
   ])


 
  

LOGDIR='log/breakhistf2v026-nsl-v4-02' 
  
adv_grad_norm = 'infinity'

adv_config = nsl.configs.make_adv_reg_config(multiplier=0.2, adv_step_size=0.05,adv_grad_norm = adv_grad_norm)
adv_model = nsl.keras.AdversarialRegularization(resnetdense, adv_config=adv_config)
 
adv_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
  

adv_model.fit(db_train,  validation_data=  db_val , validation_steps = val_steps, epochs=100, verbose =1)

images3, labels3, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'),  "tf2breakhisnewCSV",False,'test')


(x_test , y_test)  = preprocessnsl(images3 ,labels3 ) 
db_test =  tf.data.Dataset.from_tensor_slices({'feature':x_test, 'label':y_test}).batch(batch_size)
 
adv_model.evaluate( db_test)
 
  

 
 
 
 

2.0.0
4745
2.0.0
1582
Epoch 1/100
297/297 [==============================] - 424s 1s/step - loss: 1.9666 - sparse_categorical_crossentropy: 1.6115 - sparse_categorical_accuracy: 0.4844 - adversarial_loss: 1.7754 - val_loss: 0.0000e+00 - val_sparse_categorical_crossentropy: 0.0000e+00 - val_sparse_categorical_accuracy: 0.0000e+00 - val_adversarial_loss: 0.0000e+00
Epoch 2/100
297/297 [==============================] - 280s 943ms/step - loss: 1.7696 - sparse_categorical_crossentropy: 1.4314 - sparse_categorical_accuracy: 0.5135 - adversarial_loss: 1.6935 - val_loss: 1.7286 - val_sparse_categorical_crossentropy: 1.3767 - val_sparse_categorical_accuracy: 0.5183 - val_adversarial_loss: 1.6622
Epoch 3/100
297/297 [==============================] - 271s 911ms/step - loss: 1.6679 - sparse_categorical_crossentropy: 1.3336 - sparse_categorical_accuracy: 0.5322 - adversarial_loss: 1.6721 - val_loss: 1.6802 - val_sparse_categorical_crossentropy: 1.3266 - val_sparse_categorical_accuracy: 0.5278 - v

Epoch 50/100
297/297 [==============================] - 281s 947ms/step - loss: 0.7145 - sparse_categorical_crossentropy: 0.4581 - sparse_categorical_accuracy: 0.8288 - adversarial_loss: 1.2827 - val_loss: 1.3794 - val_sparse_categorical_crossentropy: 0.9006 - val_sparse_categorical_accuracy: 0.6858 - val_adversarial_loss: 2.3163
Epoch 51/100
297/297 [==============================] - 271s 912ms/step - loss: 0.7180 - sparse_categorical_crossentropy: 0.4602 - sparse_categorical_accuracy: 0.8289 - adversarial_loss: 1.2881 - val_loss: 1.4222 - val_sparse_categorical_crossentropy: 0.9471 - val_sparse_categorical_accuracy: 0.6650 - val_adversarial_loss: 2.2954
Epoch 52/100
297/297 [==============================] - 286s 962ms/step - loss: 0.6985 - sparse_categorical_crossentropy: 0.4457 - sparse_categorical_accuracy: 0.8358 - adversarial_loss: 1.2692 - val_loss: 1.4528 - val_sparse_categorical_crossentropy: 0.9515 - val_sparse_categorical_accuracy: 0.6713 - val_adversarial_loss: 2.4248
Epoc

Epoch 75/100
297/297 [==============================] - 235s 792ms/step - loss: 0.5580 - sparse_categorical_crossentropy: 0.3321 - sparse_categorical_accuracy: 0.8800 - adversarial_loss: 1.1269 - val_loss: 1.6872 - val_sparse_categorical_crossentropy: 1.1153 - val_sparse_categorical_accuracy: 0.6593 - val_adversarial_loss: 2.7650
Epoch 76/100
297/297 [==============================] - 259s 873ms/step - loss: 0.5823 - sparse_categorical_crossentropy: 0.3539 - sparse_categorical_accuracy: 0.8715 - adversarial_loss: 1.1397 - val_loss: 1.6339 - val_sparse_categorical_crossentropy: 1.0840 - val_sparse_categorical_accuracy: 0.6688 - val_adversarial_loss: 2.6581
Epoch 77/100
297/297 [==============================] - 250s 842ms/step - loss: 0.5589 - sparse_categorical_crossentropy: 0.3341 - sparse_categorical_accuracy: 0.8801 - adversarial_loss: 1.1217 - val_loss: 1.5821 - val_sparse_categorical_crossentropy: 1.0381 - val_sparse_categorical_accuracy: 0.6719 - val_adversarial_loss: 2.6309
Epoc

Epoch 100/100
297/297 [==============================] - 255s 859ms/step - loss: 0.4692 - sparse_categorical_crossentropy: 0.2665 - sparse_categorical_accuracy: 0.9061 - adversarial_loss: 1.0111 - val_loss: 1.6086 - val_sparse_categorical_crossentropy: 1.0461 - val_sparse_categorical_accuracy: 0.6700 - val_adversarial_loss: 2.7216
2.0.0
1582
50/50 [==============================] - 15s 307ms/step - loss: 1.4505 - sparse_categorical_crossentropy: 0.9517 - sparse_categorical_accuracy: 0.6947 - adversarial_loss: 2.4937


[1.4504812788963317, 0.95174, 0.6946903, 2.493707]